# **BCIE Open Data (CKAN) — Extracción y Preparación de Datos Aprobaciones de Préstamos**

**Objetivo.** Construir un pipeline modular para:
 1) Conectarse a la API de CKAN del BCIE,  
 2) Descargar **un** recurso del grupo **Aprobaciones**,
 3) Normalizar/Tipar columnas (fechas, numéricos, categóricas),  
 4) Guardar salidas (CSV/Parquet),  

**Recurso (resource_id):**
- **Aprobaciones (General)**: `9202bb58-8717-4ca1-83a0-b040d5cf5398`

**Notas de autenticación**:
- Si el recurso es público: no necesitas token (`API_TOKEN=None`).
- Si requiere autenticación: define `API_TOKEN` como variable de entorno o en un archivo `.env` (ver celda de Configuración).

**Salida esperada** (carpeta `./results`):
- `aprobaciones_prestamos.csv`
- `aprobaciones_prestamos.parquet`

In [ ]:
import os
import sys
import warnings
import unicodedata
from typing import Optional, Dict, Any, List
import pandas as pd
import numpy as np
import requests

## PASO 1: CONFIGURACIÓN INICIAL


In [ ]:
print("--- Iniciando Paso 1: Configuración ---")

# --- Silenciar warnings (opcional pero útil) ---
warnings.filterwarnings("ignore", category=UserWarning, module=r"ckanapi")

# URL del portal público de datos abiertos del BCIE
CKAN_BASE = "https://datosabiertos.bcie.org"
# No se requiere token para este recurso público
API_TOKEN = None

# Resource ID específico para "Préstamos – Aprobaciones (General)"
# (Tomado de tu notebook)
RID_PRESTAMOS = "ce88a753-57f5-4266-a57e-394600c8435d"

# Directorio de salida para los archivos generados
OUTDIR = "results"
os.makedirs(OUTDIR, exist_ok=True)

print(f"Directorio de salida: {OUTDIR}")
print(f"ID del Recurso: {RID_PRESTAMOS}")
print(f"API Base: {CKAN_BASE}")

## PASO 2: FUNCIÓN DE EXTRACCIÓN DE DATOS (API)

In [ ]:
print("\n--- Iniciando Paso 2: Extracción de Datos (API) ---")

base = CKAN_BASE.rstrip("/")
url = f"{base}/api/3/action/datastore_search"

chunk = 50000
params_common = {"resource_id": RID_PRESTAMOS, "limit": chunk}
rows = []
offset = 0
total = None

headers = {"User-Agent": "bcie-python-client/1.0"}
if API_TOKEN:
    headers["Authorization"] = API_TOKEN

print(f"Conectando a API para resource_id: {RID_PRESTAMOS}...")

while True:
    payload = params_common | {"offset": offset}

    try:
        resp = requests.get(url, params=payload, headers=headers, timeout=60)
        resp.raise_for_status()
        data = resp.json()

        if not data.get("success"):
            raise RuntimeError(f"CKAN error: {data}")

        result = data["result"]
        recs = result["records"]
        rows.extend(recs)

        if total is None:
            total = result["total"]
            print(f"Total registros: {total:,}")

        if not recs or offset + chunk >= total:
            break

        offset += chunk
        print(f"Descargados: {len(rows):,}/{total:,}")

    except requests.RequestException as e:
        print(f"Error API: {e}", file=sys.stderr)
        break
    except Exception as e:
        print(f"Error procesamiento: {e}", file=sys.stderr)
        break

print(f"\nDescarga completa: {len(rows):,} filas")

df = pd.DataFrame(rows)

print("\n" + "="*60)
print("INFO DATASET")
print("="*60)
print(f"Shape: {df.shape}")
print(f"Columnas: {list(df.columns)}")
print(f"Memoria: {df.memory_usage(deep=True).sum()/1024**2:.1f} MB")

print("\n" + "="*60)
print("ÚLTIMOS 10 REGISTROS")
print("="*60)
print(df.tail(10).to_string(index=False))

csv_path = f"{OUTDIR}/{RID_PRESTAMOS}.csv"
df.to_csv(csv_path, index=False, encoding="utf-8")
print(f"\nGuardado en: {csv_path}")
print("\n¡Listo!")

## PASO 3: FUNCIONES DE LIMPIEZA Y TIPADO


In [ ]:
print("\n--- Iniciando Paso 3 (parte 1): Limpieza de Columnas ---")

df = df.copy()

new_cols = []
for c in df.columns:
    s = str(c).strip()
    s_norm = unicodedata.normalize("NFKD", s)
    s_ascii = "".join(ch for ch in s_norm if not unicodedata.combining(ch))
    s_lower = s_ascii.lower()
    s_unders = s_lower.replace(" ", "_").replace("-", "_")
    s_clean = "".join(ch for ch in s_unders if ch.isalnum() or ch == '_')
    if s_clean == "id" and "_id" in s_lower:
        s_clean = "_id"
    new_cols.append(s_clean)

df.columns = new_cols

print("Columnas normalizadas:")
print(list(df.columns))

if 'pais' in df.columns:
    print("Columna 'pais' encontrada exitosamente.")
else:
    print(f"ADVERTENCIA: 'pais' NO encontrada. Columnas: {list(df.columns)}", file=sys.stderr)

print("\n" + "="*60)
print("ÚLTIMOS 10 REGISTROS (columnas limpias)")
print("="*60)
print(df.tail(10).to_string(index=False))

csv_clean = f"{OUTDIR}/{RID_PRESTAMOS}_limpio.csv"
df.to_csv(csv_clean, index=False, encoding="utf-8")
print(f"\nGuardado limpio en: {csv_clean}")
print("\n¡Paso 3 completado!")

In [ ]:
print("\n--- Iniciando Paso 3 (parte 2): Coerción de Tipos ---")

df = df.copy()

if "anio_aprobacion" in df.columns:
    df["anio_aprobacion"] = pd.to_numeric(df["anio_aprobacion"], errors="coerce").astype("Int64")

if "monto_bruto_usd" in df.columns:
    df["monto_bruto_usd"] = pd.to_numeric(df["monto_bruto_usd"], errors="coerce").astype(float)

if "cantidad_aprobaciones" in df.columns:
    df["cantidad_aprobaciones"] = pd.to_numeric(df["cantidad_aprobaciones"], errors="coerce").astype("Int64")

print("Tipos aplicados:")
print(df.dtypes)

print("\n" + "="*60)
print("INFO DATASET (tipos corregidos)")
print("="*60)
print(f"Shape: {df.shape}")
print(f"Memoria: {df.memory_usage(deep=True).sum()/1024**2:.1f} MB")

print("\n" + "="*60)
print("ÚLTIMOS 10 REGISTROS")
print("="*60)
print(df.tail(10).to_string(index=False))

csv_final = f"{OUTDIR}/{RID_PRESTAMOS}_final.csv"
df.to_csv(csv_final, index=False, encoding="utf-8")
print(f"\nGuardado final en: {csv_final}")
print("\n¡Paso 3 completado al 100%!")

## PASO 4: FUNCIONES DE CLASIFICACIÓN Y AGREGACIÓN


In [ ]:
print("\n--- Iniciando Paso 4 (parte 1): Clasificación de Socios ---")

def _norm_txt(s):
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s_norm = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s_norm if not unicodedata.combining(ch))

FUNDADORES = {_norm_txt(x) for x in ["Guatemala", "El Salvador", "Honduras", "Nicaragua", "Costa Rica"]}
REG_NO_F   = {_norm_txt(x) for x in ["República Dominicana", "Panamá", "Belice"]}
EXTRAREG   = {_norm_txt(x) for x in ["México", "República de China (Taiwán)", "Argentina", "Colombia", "España", "Cuba", "República de Corea"]}

def clasificar_socio(row):
    pais_norm = _norm_txt(row["pais"])
    sector_norm = _norm_txt(row["sector_institucional"])

    if pais_norm == "regional":
        return "Regionales"
    if pais_norm == "institucional" or sector_norm == "institucional":
        return "Institucionales"
    if pais_norm in FUNDADORES:
        return "Fundadores"
    if pais_norm in REG_NO_F:
        return "Regional No Fundadores"
    if pais_norm in EXTRAREG:
        return "Extraregionales"
    if pd.isna(row["pais"]) or pais_norm in {"", "sin pais"}:
        return "Sin País"
    return "Otros"

df["tipo_socio"] = df.apply(clasificar_socio, axis=1)

df = df[["_id", "anio_aprobacion", "sector_institucional", "pais", "tipo_socio", "monto_bruto_usd", "cantidad_aprobaciones"]]

print("Clasificación 'tipo_socio' aplicada y columnas reordenadas.")
print("\nDistribución de tipo_socio:")
print(df["tipo_socio"].value_counts())

print("\n" + "="*80)
print("ÚLTIMOS 10 REGISTROS (orden solicitado)")
print("="*80)
print(df.tail(10).to_string(index=False))

csv_clasif = f"{OUTDIR}/{RID_PRESTAMOS}_clasificado.csv"
df.to_csv(csv_clasif, index=False, encoding="utf-8")
print(f"\nGuardado con clasificación y orden en: {csv_clasif}")
print("\n¡Paso 4 (parte 1) completado!")

In [ ]:
print("\n--- Iniciando Paso 4 (parte 2): Agregación, Promedio y Formato Final ---")

df_agg = (
    df.groupby(["anio_aprobacion", "tipo_socio", "pais", "sector_institucional"], as_index=False)
      .agg(
          monto_total_usd_aprobados=("monto_bruto_usd", "sum"),
          cantidad_total_aprobados=("cantidad_aprobaciones", "sum")
      )
)

df_agg["promedio_aprobacion_usd"] = (
    df_agg["monto_total_usd_aprobados"] / df_agg["cantidad_total_aprobados"]
).where(df_agg["cantidad_total_aprobados"] > 0, np.nan)

# Orden descendente por año (más reciente primero)
df_agg = df_agg.sort_values("anio_aprobacion", ascending=False)

df_agg = df_agg.rename(columns={
    "anio_aprobacion": "Año",
    "tipo_socio": "Tipo de Socio",
    "pais": "País",
    "sector_institucional": "Sector Institucional",
    "monto_total_usd_aprobados": "Monto Total (USD)",
    "cantidad_total_aprobados": "Cantidad Total",
    "promedio_aprobacion_usd": "Promedio por Aprobación (USD)"
})

df_agg = df_agg[[
    "Año", "Sector Institucional", "País", "Tipo de Socio",
    "Monto Total (USD)", "Cantidad Total", "Promedio por Aprobación (USD)"
]]

# Formateo bonito
def fmt(x):
    if pd.isna(x):
        return ""
    return f"{x:,.0f}"

styled = df_agg.head(10).style.format({
    "Monto Total (USD)": fmt,
    "Cantidad Total": "{:.0f}",
    "Promedio por Aprobación (USD)": fmt
}).set_properties(**{
    'text-align': 'center',
    'font-family': 'Arial',
    'font-size': '11pt'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2c3e50'), ('color', 'white'), ('font-weight', 'bold')]},
    {'selector': 'td', 'props': [('border', '1px solid #ddd')]},
    {'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]}
])

print("Agregación completa con promedio calculado.")
print(f"\nTotal de filas agregadas: {len(df_agg):,}")
print(f"Años cubiertos: {df_agg['Año'].max()} - {df_agg['Año'].min()} (orden descendente)")

print("\n" + "="*120)
print("TOP 10 REGISTROS MÁS RECIENTES ")
print("="*120)
display(styled)

csv_final = f"{OUTDIR}/tabla_final.csv"
excel_final = f"{OUTDIR}/tabla_final.xlsx"
parquet_final = f"{OUTDIR}/tabla_final.parquet"

df_agg.to_csv(csv_final, index=False, encoding="utf-8")
df_agg.to_excel(excel_final, index=False, engine="openpyxl")
df_agg.to_parquet(parquet_final, index=False)

print(f"\n¡TODO LISTO! Archivos generados (ordenados por año descendente):")
print(f"   CSV     → {csv_final}")
print(f"   Excel   → {excel_final}")
print(f"   Parquet → {parquet_final}")

print("\n¡Paso 4 completado al 100%! Listo para Power BI, Tableau o Python.")

In [ ]:
# =============================================================================
# PHASE 0: ALIGNMENT & DATA PREPARATION
# =============================================================================
import logging
import pandas as pd
import os
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Initializing Prophet Optimization Pipeline...")
OUTDIR = "results"
os.makedirs(OUTDIR, exist_ok=True)

# Load cleaned data (Same input as XGBoost MAX)
# We assume 'df' from previous steps is ready, but let's ensure it's loaded from the clean CSV/Parquet if needed.
# For this notebook flow, we use the 'df_agg' created in the previous cell.
df_ml = df_agg.copy()

# Align Column Names for Prophet (ds, y)
# Prophet needs 'ds' (date) and 'y' (value).
# Our 'Año' is an integer, we need to convert it to a year-end date or year-start.
# Standard convention: Dec 31st of that year or Jan 1st.
# Let's use Dec 31st to match financial closing.
df_ml['ds'] = pd.to_datetime(df_ml['Año'].astype(str) + '-12-31')
df_ml['y'] = df_ml['Monto Total (USD)']

group_cols = ["País", "Sector Institucional", "Tipo de Socio"]
groups = df_ml[group_cols].drop_duplicates()

logging.info(f"Data prepared. Total Groups: {len(groups)}")


In [ ]:
# =============================================================================
# PHASE 1 & 2 (OPTIMIZADO PARA DATOS ANUALES):
# HYPERPARAMETER TUNING & RIGOROUS BACKTESTING
# =============================================================================
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.model_selection import ParameterGrid
import logging
import numpy as np
import pandas as pd

logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)

# -------------------------------------------------------------------------
# 1. GRID DE HIPERPARÁMETROS (MISMO NOMBRE: param_grid)
# -------------------------------------------------------------------------
param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.5],
    'seasonality_prior_scale': [1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

grid = list(ParameterGrid(param_grid))
print(f"Total de combinaciones a evaluar: {len(grid)}")

# -------------------------------------------------------------------------
# 2. TOP GROUPS (MISMO NOMBRE: top_groups)
#    seguimos usando los 5 primeros como en tu versión original
# -------------------------------------------------------------------------
top_groups = (
    df_ml
    .groupby(group_cols)['y']
    .sum()
    .nlargest(5)
    .reset_index()[group_cols]
)

print("Grupos usados para tuning (Top por volumen):")
print(top_groups)

# -------------------------------------------------------------------------
# 3. VARIABLES DE SALIDA (MISMO NOMBRE QUE TU BLOQUE ORIGINAL)
# -------------------------------------------------------------------------
results_tuning = []          # lista con info de cada combo
best_params_global = None
lowest_mae = float('inf')

logging.info("Starting Grid Search on Top Groups (Sampled for efficiency)...")

# -------------------------------------------------------------------------
# 4. GRID SEARCH + BACKTEST ADAPTADO A SERIES ANUALES
# -------------------------------------------------------------------------
for i, params in enumerate(grid, start=1):
    print(f"\n=== Combo {i}/{len(grid)} ===")
    print("Parámetros:", params)

    mae_acum = 0.0
    count = 0

    for _, g_row in top_groups.iterrows():
        mask = (
            (df_ml['País'] == g_row['País']) &
            (df_ml['Sector Institucional'] == g_row['Sector Institucional']) &
            (df_ml['Tipo de Socio'] == g_row['Tipo de Socio'])
        )
        df_g = df_ml[mask][['ds', 'y']].sort_values('ds')

        # Necesitamos años suficientes para que valga la pena el CV
        if len(df_g) < 8:
            continue

        # Span temporal en años (aprox).
        span_days = (df_g['ds'].max() - df_g['ds'].min()).days
        span_years = span_days / 365.0

        # -------------------------------------------------------------
        # CONFIG DE CV EN AÑOS (CONVERTIDOS A DÍAS POR PROPHET)
        # -------------------------------------------------------------
        if span_years >= 15:
            # Series largas
            initial_years = 8      # primeros 8 años para entrenar
            period_years  = 3      # nuevo corte cada 3 años
        elif span_years >= 10:
            # Series medianas
            initial_years = 5
            period_years  = 2
        else:
            # Series cortas
            initial_years = 3
            period_years  = 2

        initial = f"{365 * initial_years} days"   # entrenamiento inicial
        period  = f"{365 * period_years} days"    # frecuencia de cortes
        horizon = "365 days"                      # horizonte = 1 año

        m = Prophet(
            **params,
            yearly_seasonality=True,    # tiene sentido con datos anuales
            weekly_seasonality=False,
            daily_seasonality=False,
            interval_width=0.80
        )

        m.fit(df_g)

        try:
            df_cv = cross_validation(
                m,
                initial=initial,
                period=period,
                horizon=horizon,
                parallel="processes"     # paraleliza cortes internos
            )
            df_p = performance_metrics(df_cv)
            mae_group = df_p['mae'].mean()
            mae_acum += mae_group
            count += 1

            print(
                f"  Grupo {g_row.to_dict()} -> MAE {mae_group:,.0f} "
                f"(span ≈ {span_years:.1f} años, initial={initial}, period={period})"
            )

        except Exception as e:
            print(f"  [WARN] Falló CV para grupo {g_row.to_dict()}: {e}")
            continue

    if count > 0:
        avg_mae = mae_acum / count

        # Mantengo results_tuning como lista de algo simple (params + MAE)
        results_tuning.append((params, avg_mae))

        print(f">>> Combo {i}: Avg MAE = {avg_mae:,.0f} (en {count} grupos)")

        if avg_mae < lowest_mae:
            lowest_mae = avg_mae
            best_params_global = params

logging.info(f"\nBest Parameters found: {best_params_global}")
print("\n=======================================")
print("BEST PARAMS GLOBAL:", best_params_global)
print("LOWEST AVG MAE:", f"{lowest_mae:,.0f}")


In [ ]:
# =============================================================================
# PHASE 3: FINAL FORECAST (2026-2030) & GOVERNANCE
# =============================================================================
logging.info("Generating Final Forecasts (2026-2030) with Optimized Parameters...")

final_forecasts = []
PROPHET_VERSION = "prophet_bcie_optimizado_v1.0"

params_to_use = best_params_global if best_params_global else {
    'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 10.0, 'seasonality_mode': 'multiplicative'
}

for _, row in groups.iterrows():
    pais = row['País']
    sector = row['Sector Institucional']
    tipo = row['Tipo de Socio']
    
    mask = (df_ml['País'] == pais) & (df_ml['Sector Institucional'] == sector) & (df_ml['Tipo de Socio'] == tipo)
    df_group = df_ml[mask][['ds', 'y']]
    
    if len(df_group) < 2:
        continue
        
    m = Prophet(**params_to_use, yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    m.fit(df_group)
    
    # Forecast horizon: 5 years (2026-2030)
    future = m.make_future_dataframe(periods=5, freq='Y')
    forecast = m.predict(future)
    
    # Extract only future years
    forecast['Año'] = forecast['ds'].dt.year
    future_years = forecast[forecast['Año'] > 2025].copy()
    
    # Add segment info
    future_years['País'] = pais
    future_years['Sector Institucional'] = sector
    future_years['Tipo de Socio'] = tipo
    
    # Rename for standard schema
    future_years = future_years.rename(columns={
        'yhat': 'Prediccion',
        'yhat_lower': 'Prediccion_Baja',
        'yhat_upper': 'Prediccion_Alta'
    })
    
    final_forecasts.append(future_years[['Año', 'País', 'Sector Institucional', 'Tipo de Socio', 'Prediccion', 'Prediccion_Baja', 'Prediccion_Alta']])

df_forecast_prophet = pd.concat(final_forecasts, ignore_index=True)
df_forecast_prophet['Monto Total (USD)'] = df_forecast_prophet['Prediccion'] # Main forecast

# Governance Artifacts
csv_path = f"{OUTDIR}/forecast_{PROPHET_VERSION}.csv"
df_forecast_prophet.to_csv(csv_path, index=False)

import json
with open(f"{OUTDIR}/best_params_{PROPHET_VERSION}.json", "w") as f:
    json.dump(params_to_use, f, indent=4)

logging.info(f"Forecast generated. Saved to {csv_path}")
